In [35]:
from datasets import load_dataset
import pandas as pd
from collections import defaultdict
from tqdm import tqdm
import sys
sys.path.append('..')
from google_nlg.ser import get_ser_slots

In [4]:
SGD_dataset = load_dataset('gem', 'schema_guided_dialog')

Reusing dataset gem (/home/jc2124/.cache/huggingface/datasets/gem/schema_guided_dialog/1.1.0/a94b3f239aacc2081ffa3ea4414c3cddff25bb680c09756a27ec3e76547187b4)


  0%|          | 0/10 [00:00<?, ?it/s]

In [98]:
entity_set = defaultdict(set)
entity_count_map = defaultdict(dict)
permissible_slots = get_ser_slots("../data/dstc8-schema-guided-dialogue/")
for split in ['train', 'test', 'validation']:
    for example in tqdm(SGD_dataset[split]):
        for action in example['dialog_acts']:
            if action['slot'] in permissible_slots[example['service']]:
                for value in action['values']:
                    value = value.lower()
                    entity_set[split].add(value)
                    entity_count_map[split][value] = entity_count_map[split][value]+1 if value in entity_count_map[split] else 1

100%|██████████| 10000/10000 [00:01<00:00, 5642.14it/s]


In [101]:
len(entity_set['test'])

2738

In [102]:
sorted(entity_count_map['test'].items(), key=lambda x:x[1],reverse=True)[:30]

[('march 1st', 166),
 ('san francisco', 125),
 ('march 5th', 111),
 ('march 2nd', 110),
 ('march 9th', 102),
 ('march 6th', 102),
 ('march 10th', 100),
 ('march 4th', 100),
 ('march 11th', 97),
 ('march 3rd', 95),
 ('march 12th', 93),
 ('march 7th', 91),
 ('march 14th', 79),
 ('5 pm', 77),
 ('march 8th', 74),
 ('march 13th', 72),
 ('san jose', 58),
 ('los angeles', 48),
 ('7:30 pm', 46),
 ('new york', 46),
 ('1512 barber shop', 45),
 ('4:30 pm', 42),
 ('7 pm', 41),
 ('santa rosa', 39),
 ('6 pm', 39),
 ('6:30 pm', 39),
 ('4.3', 38),
 ('anaheim', 37),
 ('4.4', 36),
 ('union station', 36)]

In [103]:
test_unique_entities = entity_set['test'] - entity_set['train']

In [104]:
test_unique_entity_count = {k: v for k,v in entity_count_map['test'].items() if k in test_unique_entities}

In [105]:
sorted(test_unique_entity_count.items(), key=lambda x:x[1],reverse=True)[:100]

[('alfa romeo stelvio', 28),
 ('after the wedding', 26),
 ('audi a8', 25),
 ('the angry birds movie 2', 24),
 ('hustlers', 24),
 ('it chapter two', 24),
 ('good boys', 23),
 ('buick regal sportback', 23),
 ('brittany runs a marathon', 22),
 ('ad astra', 22),
 ('the lord of the rings: the return of the king', 21),
 ('blinded by the light', 21),
 ('north hollywood station', 19),
 ('fiddler: a miracle of miracles', 18),
 ('barskyex bryna g', 18),
 ('angel has fallen', 18),
 ('once upon a time in hollywood', 18),
 ('a faithful man', 16),
 ('labyrinth', 15),
 ('hobbs and shaw', 14),
 ('abbi jacobson', 14),
 ('new alarm', 14),
 ('steel magnolias', 14),
 ('penn station', 14),
 ('acting for teens', 13),
 ('pulp fiction', 13),
 ('toy story 4', 13),
 ('10th and filbert', 13),
 ('in fabric', 12),
 ('bert epstein', 12),
 ('adam e. pollock', 12),
 ('kaufmann concert hall', 11),
 ('a right royale tea', 11),
 ('9/11 tribute museum', 11),
 ('yesterday', 11),
 ('chhichhore', 11),
 ('advanced acting sce

In [106]:
len(entity_set['train'])

12788

In [107]:
len(test_unique_entity_count)

1102

In [108]:
sum(test_unique_entity_count.values())

2607

In [109]:
sum(entity_count_map['test'].values())

10166

In [111]:
unseen_diag_count = 0
domain_unseen_diag_count = defaultdict(int)
domain_unseen_occur_count = defaultdict(int)
domain_unseen_entities = defaultdict(set)
for example in tqdm(SGD_dataset['test']):
    for action in example['dialog_acts']:
        if action['slot'] in permissible_slots[example['service']]:
            unseen = False
            for value in action['values']:
                value = value.lower()
                if value in test_unique_entities:
                    domain = example['service'].split('_')[0]
                    if not unseen:
                        unseen_diag_count += 1
                        domain_unseen_diag_count[domain] += 1
                        unseen = True
                    domain_unseen_occur_count[domain] += 1
                    domain_unseen_entities[domain].add(value)
                    
                

100%|██████████| 10000/10000 [00:01<00:00, 5022.89it/s]


In [112]:
sorted(domain_unseen_occur_count.items(), key=lambda x: x[1], reverse=True)

[('Services', 386),
 ('Events', 373),
 ('Movies', 362),
 ('RentalCars', 265),
 ('Media', 248),
 ('Hotels', 158),
 ('Messaging', 124),
 ('Music', 105),
 ('Buses', 104),
 ('Alarm', 86),
 ('Restaurants', 82),
 ('Travel', 81),
 ('Homes', 75),
 ('RideSharing', 63),
 ('Payment', 51),
 ('Flights', 41),
 ('Trains', 2),
 ('Weather', 1)]

In [113]:
sorted(domain_unseen_diag_count.items(), key=lambda x: x[1], reverse=True)

[('Services', 386),
 ('Events', 373),
 ('RentalCars', 265),
 ('Movies', 252),
 ('Media', 195),
 ('Hotels', 158),
 ('Messaging', 124),
 ('Music', 105),
 ('Buses', 104),
 ('Alarm', 86),
 ('Restaurants', 82),
 ('Travel', 81),
 ('Homes', 75),
 ('RideSharing', 63),
 ('Payment', 51),
 ('Flights', 41),
 ('Trains', 2),
 ('Weather', 1)]

In [116]:
domain_unseen_entities['Media']

{'47 meters down: uncaged',
 'a faithful man',
 'ad astra',
 'after the wedding',
 'alexander calvert',
 'angel has fallen',
 'bjorn andresen',
 'blake clark',
 'blinded by the light',
 'britta steffenhagen',
 'brittany runs a marathon',
 'bruce willis',
 'caroline catz',
 'chhichhore',
 'chris galust',
 'christopher mann',
 'close encounters of the third kind',
 'cult',
 'dali benssalah',
 'documentary',
 'don blakely',
 'echo in the canyon',
 'elijah wood',
 'fiddler: a miracle of miracles',
 'frank oz',
 'give me liberty',
 'good boys',
 'helena holmes',
 'hobbs and shaw',
 'hustlers',
 'in fabric',
 'it chapter two',
 'james wlcek',
 'kate mckinnon',
 'labyrinth',
 'lori alan',
 'luce',
 "madeline's madeline",
 'midsommar',
 'mission mangal',
 'my big fat greek wedding',
 'naveen polishetty',
 'official secrets',
 'once upon a time in hollywood',
 'one cut of the dead',
 'paprika',
 'pavle cemerikic',
 'pulp fiction',
 'rafael federman',
 'roger mcguinn',
 'rojo',
 'scary stories t

In [117]:
unseen_diag_count / len(SGD_dataset['test'])

0.2444

In [118]:
unseen_diag_count

2444